# 02 · User Recommender (LightFM)

Doporučení nových her pro **reálného uživatele** (např. `w0nderCZ`).

Vstupy:
- `data/games_features.parquet` (z 01_feature_engineering.ipynb)
- `data/user_w0nderCZ_collection.parquet` (z 00_bgg_scraper.ipynb)

Model:
- **LightFM (WARP)** — hybridní CF s item/user featurami
- Item features: TF-IDF tagy + numerické rysy (popularita, weight, playing_time)
- User features: identity (jeden uživatel)

In [1]:
# === Setup: doinstalace potřebných balíčků (běží v aktuálním kernelu) ===
import sys, subprocess

def pip_install(pkgs):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
    except Exception as e:
        print("Pip install failed:", e)

pip_install(["lightfm", "scikit-learn", "scipy", "pandas", "pyarrow", "tqdm"])  # tiché -q

# Rychlé ověření importů
import numpy as np, pandas as pd, scipy, sklearn
from lightfm import LightFM
print("OK → LightFM; numpy", np.__version__, "| pandas", pd.__version__, "| sklearn", sklearn.__version__)


Pip install failed: Command '['C:\\Users\\xlnen\\anaconda3\\python.exe', '-m', 'pip', 'install', '-q', 'lightfm', 'scikit-learn', 'scipy', 'pandas', 'pyarrow', 'tqdm']' returned non-zero exit status 1.


ModuleNotFoundError: No module named 'lightfm'

In [ ]:
import os
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

DATA_DIR = "data"
games_path = os.path.join(DATA_DIR, "games_features.parquet")
user_collection_path = os.path.join(DATA_DIR, "user_w0nderCZ_collection.parquet")

assert os.path.exists(games_path), "Chybí data/games_features.parquet – spusť nejdřív 01_feature_engineering.ipynb"
assert os.path.exists(user_collection_path), "Chybí data/user_w0nderCZ_collection.parquet – spusť 00_bgg_scraper.ipynb"

games = pd.read_parquet(games_path)
games["tags"] = (games["categories"].fillna("") + " " + games["mechanics"].fillna("")).str.lower()
print("Počet her:", len(games))

df_col = pd.read_parquet(user_collection_path)
print("Načteno z kolekce uživatele:", len(df_col))
display(df_col.head())


## Vytvoření interakční matice
- Jeden uživatel (`w0nderCZ`) × všechny hry
- Pozitiva = hry v jeho kolekci (binary implicit feedback)

In [ ]:
users = ["w0nderCZ"]
uid = {"w0nderCZ": 0}
iid = {gid: i for i, gid in enumerate(games["bgg_id"])}

rows, cols, data = [], [], []
added = 0
for g in df_col["bgg_id"].dropna().astype(int):
    if g in iid:
        rows.append(uid["w0nderCZ"]) ; cols.append(iid[g]) ; data.append(1.0)
        added += 1
print("Interakcí (pozitiv):", added)

interactions = sparse.coo_matrix((data, (rows, cols)), shape=(len(users), len(games)))
interactions


## Item & user features
- Item: TF-IDF(tags) + numerické rysy (pop_norm, weight, playing_time)
- User: identity (1×1)

In [2]:
vec = TfidfVectorizer(token_pattern=r"[a-zA-Z\-]+")
X_items = vec.fit_transform(games["tags"])  # řidká matice

num = np.vstack([
    games.get("pop_norm", 0).fillna(0).values,
    games.get("weight", 0).fillna(0).values,
    games.get("playing_time", 0).fillna(0).values
]).T

item_feats = sparse.hstack([X_items, sparse.csr_matrix(num)], format="csr")
user_feats = sparse.identity(len(users), format="csr")
print("Item features shape:", item_feats.shape, "| User features shape:", user_feats.shape)


NameError: name 'TfidfVectorizer' is not defined

## Trénink LightFM (WARP)
- `no_components=32` (latentních faktorů)
- `epochs=20` (klidně zvedni na 50+ při větších datech)

In [ ]:
model = LightFM(loss="warp", no_components=32, learning_rate=0.05, item_alpha=1e-6, user_alpha=1e-6)
model.fit(interactions, item_features=item_feats, user_features=user_feats, epochs=20, num_threads=4)
print("Model natrénován.")


## Doporučení pro uživatele
- vyřadíme hry, které už v kolekci má
- vrátíme Top-N dle skóre modelu

In [ ]:
def recommend_user(u_name: str, k: int = 10):
    u = uid[u_name]
    scores = model.predict(u, np.arange(len(games)), user_features=user_feats, item_features=item_feats)
    known = set(iid[g] for g in df_col["bgg_id"] if g in iid)
    ranked = [i for i in np.argsort(-scores) if i not in known][:k]
    cols = ["name","min_players","max_players","playing_time","weight","categories","mechanics"]
    return games.iloc[ranked][cols]

display(recommend_user("w0nderCZ", k=10))


### Poznámky
- Když bude kolekce malá, LightFM se může přeučit na obsahové featury — to je v pořádku (hybridní přístup).
- Lze přidat váhy podle počtu `plays` nebo podle `user_rating`.
- Pro více uživatelů rozšiř seznam `users` a poskládej interakce pro každého (řádek v matici).